## Importation des différentes librairies dont nous avons besoin et ouverture du fichier de données que nous allons étudier

In [ ]:
# Import of all the libraries useful for this file
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pylab as plt
import seaborn as sns

# Reading of the CSV file for the MVP
MVP_food_data = pd.read_csv('/home/apprenant/Documents/Brief-5-Rachid-Karbiche/data/02_intermediate_data/MVP_openfoodfacts_data.csv', low_memory= False)

# Print of the current versions of our libraries
print('Matplotlib Version: ' + mpl.__version__)
print('NumPy Version: ' + np.version.full_version)
print('Python Version: ' + sys.version)
print('Seaborn Version: ' + sns.__version__)

In [ ]:
MVP_food_data.head()

In [ ]:
MVP_food_data.shape

Pour le MVP, j'ai donc conservé 16 colonnes sur lesquels j'ai travaillé. Sans plus tarder, commençons par faire le data cleaning :

## Recherche et traitement des doublons

Recherchons d'abord les doublons parfaits :

In [ ]:
duplicateRowsDF = MVP_food_data[MVP_food_data.duplicated()]
print(duplicateRowsDF)

In [ ]:
MVP_food_data.duplicated().value_counts()

On remarque donc qu'il y a 6 847 doublons parfaits dans notre database. Nous avons ici deux possibilités par rapport à la variable principale de notre database : product_name. La première est le fait que product_name est NaN, auquel cas les doublons ne nous permettront pas de l'identifier (on supprimera d'ailleurs les index contenant un product_name NaN). La second est le fait que product_name existe et qu'il n'est donc pas nécessaire de conserver les doublons. De plus, on pourra repérer plus facilement ces doublons en filtrant des colonnes telles que product_name, categories, ingredients_text et pnns_groups_1. Privilégions d'abord la second option en ne gardant qu'une occurrence de ces index.

In [ ]:
MVP_food_data.drop_duplicates(subset=['product_name', 'categories', 'ingredients_text', 'pnns_groups_1'])

In [ ]:
no_duplicates_MVP_data = MVP_food_data.drop_duplicates(subset=['product_name', 'categories', 'ingredients_text', 'pnns_groups_1'])

## Traitement des valeurs manquantes

Dans un premier temps, on va s'intéresser à product_name. Il s'agit de ne pas se précipiter en supprimant directement les index ne mentionnant aucun product_name. On sait en effet que categories ou pnns_groups_1 peuvent nous apporter des renseignements précieux afin de trouver le nom du produit.

In [ ]:
no_duplicates_MVP_data.product_name.isnull().value_counts()

Nous avons 405 product_name inconnus. Eliminons déjà ceux qu'on ne peut pas du tout identifier :

In [ ]:
no_duplicates_MVP_data.loc[(no_duplicates_MVP_data['product_name'].isnull()) & (no_duplicates_MVP_data['categories'].isnull()) & (no_duplicates_MVP_data['ingredients_text'].isnull()) & (no_duplicates_MVP_data['pnns_groups_1'].isnull())]

Cette ligne n'a aucun renseignement. Supprimons-la !

In [ ]:
removed_product_MVP_data = no_duplicates_MVP_data.drop(no_duplicates_MVP_data.index[3])

In [ ]:
removed_product_MVP_data.product_name.isnull().value_counts()

Poursuivons maintenant en vérifiant si un product_name inconnu a une catégorie :

In [ ]:
removed_product_MVP_data.loc[(removed_product_MVP_data['product_name'].isnull()) & (removed_product_MVP_data['categories'].notnull())]

Pour tous ces index, passons la valeur de categories à product_name :

In [ ]:
removed_product_MVP_data.loc[(removed_product_MVP_data['product_name'].isnull()) & (removed_product_MVP_data['categories'].notnull()), 'product_name'] = removed_product_MVP_data['categories']

Vérifions :

In [ ]:
removed_product_MVP_data.loc[(removed_product_MVP_data['product_name'].isnull()) & (removed_product_MVP_data['categories'].notnull())]

C'est ok ! Maintenant, voyons les index de product_name inconnus ayant un pnns_groups_1 :

In [ ]:
removed_product_MVP_data.loc[(removed_product_MVP_data['product_name'].isnull()) & (removed_product_MVP_data['pnns_groups_1'].notnull())]

Ici, on constate que tous les pnns_groups_1 sont mentionnés comme étant inconnus. Par contre, on voit que les ingrédients sont mentionnés et qu'on pourrait compléter notre colonne product_name par cela. Cependant, cela créerait des product_name à rallonge. On ne va donc pas y toucher, d'autant plus qu'il n'y a que trop peu de données concernées.

In [ ]:
removed_unknown_product_MVP_data = removed_product_MVP_data.dropna(subset=['product_name'])

In [ ]:
removed_unknown_product_MVP_data.product_name.isnull().value_counts()

Pour la colonne categories, je préfère ne pas y toucher parce que donner un product_name à une categorie serait moins cohérent que l'inverse. De ce fait, on va remplacer les NaN par 'Not specified'

In [ ]:
removed_unknown_product_MVP_data.loc[removed_unknown_product_MVP_data['categories'].isnull(), 'categories'] = removed_unknown_product_MVP_data['categories'].fillna('Not specified')

Idem par la colonne ingredients_text où l'on va remplacer les valeurs manquantes par 'Not specified'

In [ ]:
removed_unknown_product_MVP_data.loc[removed_unknown_product_MVP_data['ingredients_text'].isnull(), 'ingredients_text'] = removed_unknown_product_MVP_data['ingredients_text'].fillna('Not specified')

Pareil pour la colonne pnns_groups_1

In [ ]:
removed_unknown_product_MVP_data.loc[removed_unknown_product_MVP_data['pnns_groups_1'].isnull(), 'pnns_groups_1'] = removed_unknown_product_MVP_data['pnns_groups_1'].fillna('Not specified')

In [ ]:
removed_unknown_product_MVP_data.head()

Maintenant, nous allons pouvoir nous intéresser aux valeurs nutritives de ces produits. On va faire un premier tri au niveau de nos index où il n'y a aucun renseignement sur les valeurs nutritives. On va donc supprimer toutes celles qui n'ont tout simplement aucune donnée chiffrée.

In [ ]:
removed_unknown_product_MVP_data.columns

In [ ]:
checked_columns = ['nutrition_grade_fr', 'energy_100g', 'fat_100g',
       'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g',
       'salt_100g', 'sodium_100g', 'fruits-vegetables-nuts_100g',
       'nutrition-score-fr_100g']
removed_unknown_product_MVP_data.loc[:,checked_columns].isnull()

In [ ]:
removed_unknown_product_MVP_data = removed_unknown_product_MVP_data.dropna(subset=checked_columns, how='all')

In [ ]:
removed_unknown_product_MVP_data.shape

24 803 index sont donc concernés. On sait donc qu'ils ne nous seront d'aucune utilité dans la mesure où on ne pourra rien en tirer. Nous allons donc les supprimer.

Maintenant, nous allons reprendre les lignes où les valeurs nutritives n'existent pas ou ne sont pas mentionnées afin de les remplacer par 0.

In [ ]:
nutritive_columns = ['energy_100g', 'fat_100g',
       'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g',
       'salt_100g', 'sodium_100g', 'fruits-vegetables-nuts_100g']
removed_unknown_product_MVP_data[nutritive_columns] = removed_unknown_product_MVP_data[nutritive_columns].fillna(0)

In [ ]:
removed_unknown_product_MVP_data.head()

## Consistance, reformatage et valeurs aberrantes

Observons certaines variables et leur comportement. Dans un premier temps, on va prendre un échantillon de nos données afin d'accélérer l'exécution de notre code, notamment nos valeurs nutritives :

In [ ]:
nutritive_columns = removed_unknown_product_MVP_data[['energy_100g', 'fat_100g',
       'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g',
       'salt_100g', 'sodium_100g', 'fruits-vegetables-nuts_100g']].sample(n=10000)

Observons les corrélations entre nos variables :

In [ ]:
sns.pairplot(nutritive_columns)

On remarque que plusieurs pairs sont corrélées : energy_100g/fat_100g, energy_100g/carbohydrates_100g, energy_100g/sugars_100g, energy_100g/proteins_100g, salt_100g/sodium_100g. En recherchant quelques infos sur Internet, on remarque que certaines sont liées par calcul. Voyons cela de plus près :

In [ ]:
columns = ['energy_100g', 'fat_100g',
       'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g',
       'salt_100g', 'sodium_100g', 'fruits-vegetables-nuts_100g']

for i in columns:
    for j in columns:
        corr_coeff = round(removed_unknown_product_MVP_data[i].corr(removed_unknown_product_MVP_data[j]), ndigits=3)
        print('Le coefficient de corrélation entre {} et {} est de {}'.format(i, j, corr_coeff))
    print('------------------------')

Observons une première relation entre deux variables : salt_100g et sodium_100g. En effet, pour passer de l'un à l'autre, il faut réaliser une conversion qui est la suivante : salt_100g = sodium_100g * 2.54. Complétons les valeurs de ces colonnes si elles manquent :

In [ ]:
removed_unknown_product_MVP_data.loc[(removed_unknown_product_MVP_data['salt_100g'].isnull()) & (removed_unknown_product_MVP_data['sodium_100g'].notnull())]

In [ ]:
removed_unknown_product_MVP_data.loc[(removed_unknown_product_MVP_data['sodium_100g'].isnull()) & (removed_unknown_product_MVP_data['salt_100g'].notnull())]

Le sodium nous sera utile pour le calcul du Nutri-Score. Passons à la suite !

Observons la variable energy_100g. Le but est de relever si certaines valeurs ne dépassent pas une certaine limite. Celle-ci est définie à 3800 kJ car pour 100g, les éléments nutritifs apportant le plus d'énergie au gramme sont les lipides avec 38 kJ. On va donc harmoniser ces valeurs et leur attribuer la valeur seuil, à savoir 3800 kJ.

In [ ]:
fig, ax = plt.subplots(figsize=(9,6))
sns.boxplot(x=removed_unknown_product_MVP_data["energy_100g"])

On va créer une fonction (dont on se resservira) afin d'harmoniser nos valeurs à la limite

In [ ]:
def harmonize_exceeded_values(name_variable, limit):
    exceeded_variable = removed_unknown_product_MVP_data[name_variable] > limit
    focused_column = name_variable
    removed_unknown_product_MVP_data.loc[exceeded_variable, focused_column] = limit

In [ ]:
harmonize_exceeded_values('energy_100g', 3800)

Vérifions :

In [ ]:
fig, ax = plt.subplots(figsize=(9,6))
sns.boxplot(x=removed_unknown_product_MVP_data["energy_100g"])

En voyant les différents graphes, on remarque d'ailleurs que de nombreuses variables présentent des valeurs supérieures à ce qu'elles devraient avoir. Vérifions cela :

In [ ]:
identified_columns = ['product_name', 'nutrition_grade_fr', 'energy_100g', 'fat_100g',
       'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g',
       'salt_100g', 'sodium_100g', 'fruits-vegetables-nuts_100g',
       'nutrition-score-fr_100g']

In [ ]:
removed_unknown_product_MVP_data.loc[removed_unknown_product_MVP_data['fat_100g'] > 100, identified_columns]

In [ ]:
harmonize_exceeded_values('fat_100g', 100)

Vérifions :

In [ ]:
removed_unknown_product_MVP_data.loc[removed_unknown_product_MVP_data['fat_100g'] > 100, identified_columns]

In [ ]:
removed_unknown_product_MVP_data.loc[removed_unknown_product_MVP_data['saturated-fat_100g'] > 100, identified_columns]

In [ ]:
removed_unknown_product_MVP_data.loc[removed_unknown_product_MVP_data['carbohydrates_100g'] > 100, identified_columns]

In [ ]:
harmonize_exceeded_values('carbohydrates_100g', 100)

Vérifions :

In [ ]:
removed_unknown_product_MVP_data.loc[removed_unknown_product_MVP_data['carbohydrates_100g'] > 100, identified_columns]

In [ ]:
removed_unknown_product_MVP_data.loc[removed_unknown_product_MVP_data['sugars_100g'] > 100, identified_columns]

In [ ]:
harmonize_exceeded_values('sugars_100g', 100)

Vérifions :

In [ ]:
removed_unknown_product_MVP_data.loc[removed_unknown_product_MVP_data['sugars_100g'] > 100, identified_columns]

In [ ]:
removed_unknown_product_MVP_data.loc[removed_unknown_product_MVP_data['fiber_100g'] > 100, identified_columns]

In [ ]:
harmonize_exceeded_values('fiber_100g', 100)

Vérifions :

In [ ]:
removed_unknown_product_MVP_data.loc[removed_unknown_product_MVP_data['fiber_100g'] > 100, identified_columns]

In [ ]:
removed_unknown_product_MVP_data.loc[removed_unknown_product_MVP_data['proteins_100g'] > 100, identified_columns]

In [ ]:
harmonize_exceeded_values('proteins_100g', 100)

Vérifions :

In [ ]:
removed_unknown_product_MVP_data.loc[removed_unknown_product_MVP_data['proteins_100g'] > 100, identified_columns]

In [ ]:
removed_unknown_product_MVP_data.loc[removed_unknown_product_MVP_data['salt_100g'] > 100, identified_columns]

In [ ]:
removed_unknown_product_MVP_data.loc[removed_unknown_product_MVP_data['sodium_100g'] > 100, identified_columns]

In [ ]:
removed_unknown_product_MVP_data.loc[removed_unknown_product_MVP_data['fruits-vegetables-nuts_100g'] > 100, identified_columns]